## Tests

Tests for `configure_logging()` - the one-time logging configuration function.

In [ ]:
#| hide
import logging
import tempfile
from pathlib import Path
from fh_saas.utils_log import configure_logging

### Test 1: Basic Console Logging Setup

In [ ]:
#| hide
print("\n1️⃣ Testing basic console logging setup...")

# Configure with INFO level
configure_logging(level='INFO')

# Get the package logger
logger = logging.getLogger('fh_saas')

# Verify logger configuration
assert logger.level == logging.INFO, "Logger level should be INFO"
assert len(logger.handlers) >= 1, "Should have at least console handler"
assert isinstance(logger.handlers[0], logging.StreamHandler), "First handler should be StreamHandler"
assert logger.propagate is False, "Should not propagate to root logger"

print("   ✅ Console logging configured correctly")
print(f"   ✅ Log level: {logging.getLevelName(logger.level)}")
print(f"   ✅ Handlers: {len(logger.handlers)}")


1️⃣ Testing basic console logging setup...
   ✅ Console logging configured correctly
   ✅ Log level: INFO
   ✅ Handlers: 1


### Test 2: File Logging with Rotation

In [ ]:
#| hide
print("\n2️⃣ Testing file logging with rotation...")

# Create temporary log file
with tempfile.TemporaryDirectory() as tmpdir:
    log_file = Path(tmpdir) / 'test.log'
    
    # Configure with file logging
    configure_logging(log_file=str(log_file), level='DEBUG', max_bytes=1000, backup_count=3)
    
    # Get logger and verify configuration
    logger = logging.getLogger('fh_saas')
    assert len(logger.handlers) == 2, "Should have console + file handler"
    
    # Find the file handler
    file_handler = None
    for handler in logger.handlers:
        if isinstance(handler, logging.handlers.RotatingFileHandler):
            file_handler = handler
            break
    
    assert file_handler is not None, "Should have RotatingFileHandler"
    assert file_handler.maxBytes == 1000, "Max bytes should be 1000"
    assert file_handler.backupCount == 3, "Backup count should be 3"
    
    # Write a test log
    logger.info("Test log message")
    
    # Flush and close all handlers to release file locks (Windows)
    for handler in logger.handlers[:]:
        handler.flush()
        handler.close()
        logger.removeHandler(handler)
    
    # Verify file was created and contains the message
    assert log_file.exists(), "Log file should be created"
    content = log_file.read_text()
    assert "Test log message" in content, "Log file should contain test message"
    assert "fh_saas" in content, "Log file should contain logger name"
    
    print("   ✅ File handler configured correctly")
    print(f"   ✅ Log file created: {log_file.name}")
    print(f"   ✅ Rotation: max_bytes=1000, backup_count=3")

2026-01-15 23:22:24 | fh_saas | INFO | Test log message



2️⃣ Testing file logging with rotation...
   ✅ File handler configured correctly
   ✅ Log file created: test.log
   ✅ Rotation: max_bytes=1000, backup_count=3


In [ ]:
#| hide
print("\n" + "="*60)
print("✅ ALL LOGGING TESTS PASSED!")
print("="*60)
print("\nSummary:")
print("  • Basic console logging: ✅")
print("  • File logging with rotation: ✅")
print("  • Log level configuration: ✅")
print("  • Handler setup: ✅")


✅ ALL LOGGING TESTS PASSED!

Summary:
  • Basic console logging: ✅
  • File logging with rotation: ✅
  • Log level configuration: ✅
  • Handler setup: ✅


: 